<a href="https://colab.research.google.com/github/faisalfaturohman/Mall-Customer-Segmentation-Using-Clustering/blob/main/Mall_Customer_Segmentation_Using_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>